## Prerequisites

In [15]:
# %pip install tabulate

In [16]:
def df_to_org_table(df):
    print(df.to_markdown(index=False, tablefmt='grid', colalign=['left']*len(df.columns), maxcolwidths=[None, None, 40, 80]))

## Extract configs of all stages as list of Dataframes

In [17]:
from ska_sdp_spectral_line_imaging import pipeline
import pandas as pd

dfs = {}

for x in pipeline.spectral_line_imaging_pipeline._stages:
    df = pd.DataFrame([{"param": k, **v.__dict__} for k,v in x._Stage__config._Configuration__config_params.items()]).fillna("None")
    if df.empty:
        continue
    df = df.rename(columns={'_type': 'type'})
    df.columns = df.columns.str.capitalize()
    df['Type'] = df['Type'].apply(lambda x: x.__name__)
    dfs[x.name]=df

In [18]:
dfs.keys()

dict_keys(['select_vis', 'vis_stokes_conversion', 'read_model', 'predict_stage', 'export_model', 'continuum_subtraction', 'imaging', 'export_residual', 'export_image'])

In [19]:
df = dfs["vis_stokes_conversion"]
df_to_org_table(df)

+----------------------+--------+------------+---------------------------------------------------------------------------------+
| Param                | Type   | Default    | Description                                                                     |
+======================+========+============+=================================================================================+
| output_polarizations | list   | ['I', 'Q'] | List of desired polarization codes, in the order they will appear in the output |
|                      |        |            | dataset polarization axis                                                       |
+----------------------+--------+------------+---------------------------------------------------------------------------------+


## Handing imaging configuration

Since imaging stage config contains dictionaries inside each ConfigParam, the key value pairs inside dictionary have to be extracted out seperately

In [ ]:
image_df = dfs["imaging"]
image_df

In [8]:
new_rows = []
params_to_remove = []

# Iterate over the rows of the original DataFrame
for _, row in image_df.iterrows():
    param = row['Param']
    default = row['Default']

    if isinstance(default, dict):
        params_to_remove.append(param)
        # Iterate over the dictionary items in the 'Default' column
        for key, value in default.items():
            new_param = f"{param}.{key}"  # Combine the Param and key
            if value is None:
                value = "None"
            new_rows.append({'Param': new_param, 'Type': type(value).__name__, 'Default': value, 'Description': ""})

new_df = pd.DataFrame(new_rows)

for name in params_to_remove:
    image_df = image_df[image_df["Param"] != name]


image_df = pd.concat([image_df, new_df], ignore_index=True)

In [9]:
# Mannually adding missing data

image_df.loc[image_df['Param'] == 'gridding_params.cell_size', 'Type'] = "float"
image_df.loc[image_df['Param'] == 'gridding_params.cell_size', 'Description'] = "Cell size of the image in arcseconds"
image_df.loc[image_df['Param'] == 'gridding_params.scaling_factor', 'Description'] = "Scaling factor used for esimation of cell size."
image_df.loc[image_df['Param'] == 'gridding_params.epsilon', 'Description'] = "Floating point accuracy for ducc gridder"
image_df.loc[image_df['Param'] == 'gridding_params.image_size', 'Description'] = "Spatial size (nx and ny) of the spectral cube"

image_df.loc[image_df['Param'] == "deconvolution_params.algorithm", 'Description'] = "Algorithm for deconvolution. If use_radler is False, then options are 'hogbom'|'msclean'. With radler, the options are 'multiscale'|'iuwt'|'more_sane'|'generic_clean'"
image_df.loc[image_df['Param'] == "deconvolution_params.gain", 'Description'] = "Loop gain"
image_df.loc[image_df['Param'] == "deconvolution_params.threshold", 'Description'] = "Clean threshold"
image_df.loc[image_df['Param'] == "deconvolution_params.fractional_threshold", 'Description'] = "Fractional threshold"
image_df.loc[image_df['Param'] == "deconvolution_params.scales", 'Description'] = "Scales in pixels for multiscale"
image_df.loc[image_df['Param'] == "deconvolution_params.niter", 'Description'] = "Maximum number of minor cycle iterations"
image_df.loc[image_df['Param'] == "deconvolution_params.use_radler", 'Description'] = "Whether to use radler or not"

image_df.loc[image_df['Param'] == "beam_info.bmaj", 'Type'] = "float"
image_df.loc[image_df['Param'] == "beam_info.bmaj", 'Description'] = "Beam major axis in radian"
image_df.loc[image_df['Param'] == "beam_info.bmin", 'Type'] = "float"
image_df.loc[image_df['Param'] == "beam_info.bmin", 'Description'] = "Beam minor axis in radian"
image_df.loc[image_df['Param'] == "beam_info.bpa", 'Type'] = "float"
image_df.loc[image_df['Param'] == "beam_info.bpa", 'Description'] = "Beam position angle in radian"

In [ ]:
df_to_org_table(image_df)